# TorchServe 安装与运行
- 基于 Python 和 java8
```shell
haha7@carpediem:~$ sudo apt-get install openjdk-11-jdk
```
- 创建 conda 虚拟环境

```shell
# 创建虚拟环境
haha7@carpediem:~$ conda create -n torchserve
haha7@carpediem:~$ conda activate torchserve

# 安装pytorch包
(torchserve) haha7@carpediem:~$ conda install -c pytorch -c powerai pytorch torchtext torchvision

# 安装torchserve
(torchserve) haha7@carpediem:~$ conda install -c pytorch torchserve torch-model-archiver
```




- 安装，利用 docker 运行
```shell
haha7@carpediem:~$ docker run --rm -it -p 8080:8080 -p 8081:8081 pytorch/torchserve:latest
```

# 拉取并配置docker容器
```shell
# 1. 拉取 torchserve 容器
hahaha@carpediem:~$ docker pull pytorch/torchserve

# 2. 进入容器命令行界面
hahaha@carpediem:~$ docker run -it pytorch/torchserve /bin/bash

# 3. 安装必要的包
model-server@0ab89874a541:~$ pip install -f https://download.pytorch.org/whl/torch_stable.html torchserve torch-model-archiver

# 4. 将 torch-model-archiver 脚本路径添加到 PATH 中         
model-server@0ab89874a541:~$ vim ~/.bashrc
# 在文件最后一行添加：export PATH=/home/model-server/.local/bin:$PATH

# 5. 然后激活设置，可以运行命令 torch-model-archiver
model-server@0ab89874a541:~$ source ~/.bashrc
model-server@0ab89874a541:~$ torch-model-archiver
usage: torch-model-archiver [-h] --model-name MODEL_NAME --serialized-file
......

# 6. 查看容器内内容
model-server@0ab89874a541:~$ ls -al
total 52
drwxr-xr-x 1 model-server model-server 4096 May 24 09:49 .
drwxr-xr-x 1 root         root         4096 Apr 11 21:04 ..
-rw-r--r-- 1 model-server model-server  220 Apr  4  2018 .bash_logout
-rw-r--r-- 1 model-server model-server 3856 May 24 09:49 .bashrc
drwxr-xr-x 3 model-server model-server 4096 May 24 09:38 .cache
drwx------ 4 model-server model-server 4096 May 24 09:42 .local
-rw-r--r-- 1 model-server model-server  807 Apr  4  2018 .profile
-rw------- 1 model-server model-server  808 May 24 09:49 .viminfo
-rw-rw-r-- 1 root         root          166 Apr 10 22:51 config.properties
drwxr-xr-x 2 model-server root         4096 Apr 11 21:04 model-store
drwxr-xr-x 1 model-server root         4096 May 24 09:42 tmp
```

# 将模型添加到容器内

```shell
# 1. 查看上一步运行的容器的ID, 活跃的容器才能向里面添加模型
yangbin7@carpediem:~$ docker ps -a
CONTAINER ID        IMAGE                COMMAND                  CREATED             
11504561baf9  

# 2.　下载模型 `.pth` ，然后将文件上传到容器内的`model-store`文件夹，需要指明容器的ID
hahaha@carpediem:~$ docker cp densenet161-8d451a50.pth 11504561baf9:/home/model-server/model-store

```

# 创建模型脚本 `model.py`,并上传到容器内文件夹`app`

```python
from torchvision.models.densenet import DenseNet

class ImageClassifier(DenseNet):
    def __init__(self):
        super(ImageClassifier, self).__init__(48, (6, 12, 36, 24), 96)
    
    def load_state_dict(self, state_dict, strict=True):
        # '.'s are no longer allowed in module names, but previous _DenseLayer
        # has keys 'norm.1', 'relu.1', 'conv.1', 'norm.2', 'relu.2', 'conv.2'.
        # They are also in the checkpoints in model_urls. This pattern is used
        # to find such keys.
        # Credit - https://github.com/pytorch/vision/blob/master/torchvision/models/densenet.py
        # #def _load_state_dict()
        import re
        pattern = re.compile(
            r'^(.*denselayer\d+\.(?:norm|relu|conv))\.((?:[12])\.('
            r'?:weight|bias|running_mean|running_var))$')
        
        for key in list(state_dict.keys()):
            res = pattern.match(key)
            if res:
                new_key = res.group(1) + res.group(2)
                state_dict[new_key] = state_dict[key]
                del state_dict[key]
        
        return super(ImageClassifier, self).load_state_dict(state_dict, strict)
```        
上传文件:
```shell
hahaha@carpediem:~$ docker cp model.py 11504561baf9:/home/model-server/app
```
额外的文件,保存类别名:
```shell
hahaha@carpediem:~$ docker cp index_to_name.json 11504561baf9:/home/model-server/app
```

# 配置应用
```shell
# 1. 上一步运行的容器中,配置应用
model-server@11504561baf9:~$ torch-model-archiver --model-name densenet161 \
--version 1.0 \
--model-file app/model.py \
--serialized-file model-store/densenet161-8d451a50.pth \
--handler image_classifier \
--extra-files app/index_to_name.json
```

# 三种API
- TorchServe推理APIs的描述信息：
```shell
haha7@carpediem:~$ curl -X OPTIONS http://localhost:8080
{
  "code": 405,
  "type": "MethodNotAllowedException",
  "message": "Requested method is not allowed, please refer to API document."
}
```      
- TorchServe健康检查：  
```shell
haha7@carpediem:~$ curl http://localhost:8080/ping
{
  "status": "Healthy"
}
``` 
- 模型预测API    
                                   
     - 利用加载的模型的默认版本，调用 `POST /predictions/{model_name}`
```shell
haha7@carpediem:~$ curl -O https://s3.amazonaws.com/model-server/inputs/kitten.jpg
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  108k  100  108k    0     0   6127      0  0:00:18  0:00:18 --:--:-- 11575
haha7@carpediem:~$ curl -X POST http://localhost:8080/predictions/resnet-18 -T kitten.jpg
{
  "code": 404,
  "type": "ModelNotFoundException",
  "message": "Model not found: resnet-18"
}
```

 - 利用特定版本的模型,`POST /predictions/{model_name}/{version}`
```shell
 haha7@carpediem:~$ curl -X POST http://localhost:8080/predictions/resnet-18/2.0 -F "data=@kitten.jpg"
```
返回结果：
```json
{
    "class": "n02123045 tabby, tabby cat",
    "probability": 0.42514491081237793
}
```

# 预置的模型

## 图像分类器
https://github.com/pytorch/serve/tree/master/examples/image_classifier   
输入：RGB图像   
输出：概率最高的5类及对应的概率


- 将 `.pth` 模型下载到文件夹 `model_store`
- 在文件夹`app` 创建脚本 `model.py`，和分类的类别与类名的对应关系文件`index_to_name.json`

```python
from torchvision.models.densenet import DenseNet

class ImageClassifier(DenseNet):
    def __init__(self):
        super(ImageClassifier, self).__init__(48, (6, 12, 36, 24), 96)

    def load_state_dict(self, state_dict, strict=True):
        # '.'s are no longer allowed in module names, but previous _DenseLayer
        # has keys 'norm.1', 'relu.1', 'conv.1', 'norm.2', 'relu.2', 'conv.2'.
        # They are also in the checkpoints in model_urls. This pattern is used
        # to find such keys.
        # Credit - https://github.com/pytorch/vision/blob/master/torchvision/models/densenet.py#def _load_state_dict()
        import re
        pattern = re.compile(r'^(.*denselayer\d+\.(?:norm|relu|conv))\.((?:[12])\.(?:weight|bias|running_mean|running_var))$')

        for key in list(state_dict.keys()):
            res = pattern.match(key)
            if res:
                new_key = res.group(1) + res.group(2)
                state_dict[new_key] = state_dict[key]
                del state_dict[key]

        return super(ImageClassifier, self).load_state_dict(state_dict, strict)
```
- 将模型和脚本注册到TorchServe
```bash
haha7@carpediem:~$ docker run -it pytorch/torchserve /bin/bash
model-server@54121b3750a7:~$ 
```